# Programação Orientada a Objetos (POO) em Python 

[Aprenda Python com Jupyter](https://github.com/jeanto/python_programming_course_notebook) by [Jean Nunes](https://jeanto.github.io/jeannunes)   
Code license: [GNU-GPL v3](https://www.gnu.org/licenses/gpl-3.0.en.html)

---

## **Padrões de Projeto na POO**

Os padrões de projeto são soluções reutilizáveis para problemas recorrentes no desenvolvimento de software. Eles ajudam a estruturar o código de forma mais eficiente, legível e escalável.

### **1. Strategy**

O padrão __Strategy__ permite definir uma família de algoritmos, encapsulá-los em classes separadas e torná-los intercambiáveis, facilitando a manutenção e expansão do código. É o padrão ideal para implementar a lógica de determinação da gravidade da condição e, com base nisso, definir a posição na lista de espera.


#### **Requisitos**

A lógica para determinar a gravidade será baseada nas seguintes regras:

1. **Gravidade Alta**:
   - O receptor precisa de **coração**, **pulmão** ou **rins** **e** tem mais de **60 anos**.

2. **Gravidade Média**:
   - O receptor precisa de **coração**, **pulmão** ou **rins** **e** tem entre **40 e 60 anos**.

3. **Gravidade Baixa**:
   - O receptor tem menos de **40 anos** **ou** não precisa de **coração**, **pulmão** ou **rins**.

---

#### **Implementação**

**1. Interface da Estratégia**

A interface define o método `determinar_gravidade`, que será implementado por cada estratégia.


In [5]:
from abc import ABC, abstractmethod

class EstrategiaGravidade(ABC):
    @abstractmethod
    def determinar_gravidade(self, idade, orgao_necessario):
        pass

**2. Estratégias Concretas**

Cada estratégia implementa a lógica para determinar a gravidade.

**Gravidade Alta**


In [6]:
class GravidadeAlta(EstrategiaGravidade):
    def determinar_gravidade(self, idade, orgao_necessario):
        if orgao_necessario in ["coração", "pulmão", "rins"] and idade > 60:
            return "Alta"
        return None

**Gravidade Média**

In [7]:
class GravidadeMedia(EstrategiaGravidade):
    def determinar_gravidade(self, idade, orgao_necessario):
        if orgao_necessario in ["coração", "pulmão", "rins"] and 40 <= idade <= 60:
            return "Média"
        return None

**Gravidade Baixa**

In [8]:
class GravidadeBaixa(EstrategiaGravidade):
    def determinar_gravidade(self, idade, orgao_necessario):
        if idade < 40 or orgao_necessario not in ["coração", "pulmão", "rins"]:
            return "Baixa"
        return None

**3. Contexto**

O contexto utiliza uma estratégia para determinar a gravidade.


In [9]:
class ContextoGravidade:
    def __init__(self, estrategia: EstrategiaGravidade):
        self._estrategia = estrategia

    def definir_estrategia(self, estrategia: EstrategiaGravidade):
        self._estrategia = estrategia

    def determinar_gravidade(self, idade, orgao_necessario):
        return self._estrategia.determinar_gravidade(idade, orgao_necessario)

**4. Classe Receptor**

A classe Receptor utiliza o contexto para determinar a gravidade e a posição na lista de espera.

In [10]:
from abc import ABC

class Pessoa(ABC):
    def __init__(self, nome, idade, id=None):
        self._nome = nome
        self._idade = idade
        self._id = id

    def __str__(self):
        return (
            f"+{'-'*30}+{'-'*30}+\n"
            f"| {'Id:'.ljust(28)} | {str(self._id).ljust(28)} |\n"
            f"| {'Nome:'.ljust(28)} | {self._nome.ljust(28)} |\n"
            f"| {'Idade:'.ljust(28)} | {str(self._idade).ljust(28)} |\n"
            f"+{'-'*30}+{'-'*30}+"
        )

In [18]:
class Receptor(Pessoa):
    receptores = {}
    contador_id = 0
    
    def __init__(self, nome, idade, tipo_sanguineo, orgao_necessario, centro_transplante_vinculado):
        Receptor.contador_id += 1 # Incrementa o contador de IDs
        super().__init__(nome, idade, id=Receptor.contador_id)
        self._tipo_sanguineo = tipo_sanguineo
        self._orgao_necessario = orgao_necessario
        self._centro_transplante_vinculado = centro_transplante_vinculado
        
        self._gravidade_condicao = None
        self._posicao_lista_espera = None

    def __str__(self):
        info = super().__str__() + "\n"
        if self._orgao_necessario:
            info += f"{str('Órgão Necessário:').ljust(30)} {self._orgao_necessario}\n"
        if self._tipo_sanguineo:
            info += f"{str('Tipo Sanguíneo:').ljust(30)} {self._tipo_sanguineo}\n"
        if self._centro_transplante_vinculado:
            info += f"{str('Centro de Transplante:').ljust(30)} {self._centro_transplante_vinculado}\n"

        if self._gravidade_condicao:
            info += f"{str('Gravidade da Condição:').ljust(30)} {self._gravidade_condicao}\n"
        if self._posicao_lista_espera:
            info += f"{str('Posição na Lista de Espera:').ljust(30)} {self._posicao_lista_espera}\n"

        return info

    @property
    def id(self):
        return self._id

    @classmethod
    def cadastrar_receptor(cls, nome, idade, tipo_sanguineo, orgao_necessario, 
                 gravidade_condicao, centro_transplante_vinculado, 
                 posicao_lista_espera):
        """
        Método de classe para criar e retornar um objeto Receptor com os dados fornecidos.

        Args:
            nome (str): O nome do receptor.
            idade (any): A idade do receptor. Será convertida para inteiro.
            tipo_sanguineo (str): O tipo sanguíneo do receptor.

        Returns:
            Doador: Um objeto da classe Receptor com os atributos definidos, ou None se a idade for inválida.
        """
        ### Validações ###
            
        receptor = cls(nome, idade, tipo_sanguineo, orgao_necessario, centro_transplante_vinculado)
        
        cls.receptores[receptor.id] = {}

        # Salva receptor em um dicionário
        cls.receptores[receptor.id]["dados"] = {
            "nome": nome,
            "idade": idade,
            "tipo_sanguineo": tipo_sanguineo            
        }
        cls.receptores[receptor.id]["necessidade"] = {
            "orgao_necessario": orgao_necessario,
            "gravidade_condicao": gravidade_condicao, 
            "centro_transplante_vinculado": centro_transplante_vinculado, 
            "posicao_lista_espera": posicao_lista_espera
        }

        return receptor
    
    def definir_gravidade(self):
        contexto = ContextoGravidade(GravidadeAlta())
        self._gravidade_condicao = contexto.determinar_gravidade(self._idade, self._orgao_necessario)

        if not self._gravidade_condicao:
            contexto.definir_estrategia(GravidadeMedia())
            self._gravidade_condicao = contexto.determinar_gravidade(self._idade, self._orgao_necessario)

        if not self._gravidade_condicao:
            contexto.definir_estrategia(GravidadeBaixa())
            self._gravidade_condicao = contexto.determinar_gravidade(self._idade, self._orgao_necessario)

#### Carrega receptores

In [1]:
from faker_data_receptores import generate_receptores_short

In [ ]:
receptor = generate_receptores_short()

In [23]:
for _ in range(10):
    receptor = generate_receptores_short()
    novo_receptor = Receptor.cadastrar_receptor(
        nome=receptor["dados"]["nome"],
        idade=receptor["dados"]["idade"],
        tipo_sanguineo=receptor["dados"]["tipo_sanguineo"],
        orgao_necessario=str(receptor["necessidade"]["orgao_necessario"]).lower(),
        gravidade_condicao=receptor["necessidade"]["gravidade_condicao"],
        centro_transplante_vinculado=receptor["necessidade"]["centro_transplante"],
        posicao_lista_espera=receptor["necessidade"]["posicao_lista_espera"]
    )

    novo_receptor.definir_gravidade()

    # Abre o arquivo em modo append (adiciona ao final)
    with open("saida.txt", "a") as arquivo:
        print(novo_receptor, file=arquivo)
    